In [ ]:
!pip install pymssql --user

In [1]:
import pandas as pd
import pymssql

In [74]:
import json

with open('my.json') as file:
    data = json.load(file)

user = data['user']
pwd =  data['password']

In [76]:
conn=pymssql.connect(host='103.24.200.157',user=user,password=pwd,database='InnoculateNCRIDBI')

In [3]:
events=pd.read_sql("select EventId,McuId,AtTime,Msg,EventType,OpenedOn,ClosedBy,ClosedOn,qStatus from lbtEvents where OpenedOn>='2021-10-01 00:00:00' and OpenedOn<'2021-11-01 00:00:00' ",conn)

In [4]:
pd.set_option('display.max_columns',None)
events.shape

(66221, 9)

In [5]:
sitemas=pd.read_sql('Select s_Id,s_Name,McuId from lbtSiteMas',conn)
sitemas.head()

,s_Id,s_Name,McuId
0,11014,ID021112,7359fb
1,11015,ID000614,735572
2,11016,ID209011,735ab6
3,11019,ID000621,735b50
4,11020,ID021314,735eb8


In [6]:
merge1=pd.merge(events,sitemas,on='McuId')

In [7]:
merge1.shape

(66167, 11)

In [8]:
remarks=pd.read_sql("select EventId,UpdatedBy,UpdatedON,Remarks from lbtEventsRemarks where UpdatedON>='2021-10-01 00:00:00' and UpdatedON<='2021-11-01 00:00:00' ",conn)
remarks.head()

,EventId,UpdatedBy,UpdatedON,Remarks
0,950281,jayalakshmi.ezhilarasan@aparinnosys.com,2021-10-01 08:09:06.570,<br/> jayalakshmi.ezhilarasan@aparinnosys.com ...
1,951380,jayalakshmi.ezhilarasan@aparinnosys.com,2021-10-01 08:09:15.250,<br/> jayalakshmi.ezhilarasan@aparinnosys.com ...
2,954597,jayalakshmi.ezhilarasan@aparinnosys.com,2021-10-01 08:09:25.973,<br/> jayalakshmi.ezhilarasan@aparinnosys.com ...
3,956066,jayalakshmi.ezhilarasan@aparinnosys.com,2021-10-01 08:09:40.907,<br/> jayalakshmi.ezhilarasan@aparinnosys.com ...
4,957670,jayalakshmi.ezhilarasan@aparinnosys.com,2021-10-01 08:10:01.893,<br/> jayalakshmi.ezhilarasan@aparinnosys.com ...


In [9]:
merge2=pd.merge(merge1,remarks,on='EventId')

In [10]:
merge2.shape

(20541, 14)

In [22]:
merge3=merge2.drop_duplicates(keep='first')
merge3['qStatus']=merge3['qStatus'].astype(str)
merge3['qStatus']=merge3['qStatus'].str.lower()
merge4=merge3[merge3['qStatus']=='true']
merge4.shape

(20541, 14)

In [25]:
merge4['ClosedBy']=merge4['ClosedBy'].str.lower()
mergedrop=merge4[merge4['ClosedBy']=='system']
print(mergedrop.shape)
merge5=merge4.drop(mergedrop.index)
print(merge5.shape)

(12, 14)
(20529, 14)


In [27]:
merge5['Remarks']=merge5['Remarks'].str.lower()
merge55=merge5[merge5['Remarks'].str.contains('working')]
merge5drop=merge55[merge55['Remarks'].str.contains('closed')]
merge5drop.shape

(836, 14)

In [29]:
pd.set_option('display.max_rows',None)
Events=merge55.drop(merge5drop.index)
Events.shape

(8836, 14)

In [30]:
Events.head()

,EventId,McuId,AtTime,Msg,EventType,OpenedOn,ClosedBy,ClosedOn,qStatus,s_Id,s_Name,UpdatedBy,UpdatedON,Remarks
1,967527,735ab6,2021-10-04 06:18:06,Backroom door operated without authorization,Critical,2021-10-04 06:17:34.570,prasanth.loganathan@aparinnosys.com,2021-10-04 06:22:33.933,true,11016,ID209011,prasanth.loganathan@aparinnosys.com,2021-10-04 06:19:20.263,<br/> prasanth.loganathan@aparinnosys.com working
2,967527,735ab6,2021-10-04 06:18:06,Backroom door operated without authorization,Critical,2021-10-04 06:17:34.570,prasanth.loganathan@aparinnosys.com,2021-10-04 06:22:33.933,true,11016,ID209011,prasanth.loganathan@aparinnosys.com,2021-10-04 06:19:20.360,<br/> prasanth.loganathan@aparinnosys.com working
4,969055,735ab6,2021-10-04 17:19:44,ATM-1 Chest Door operated without authorization,Critical,2021-10-04 17:19:10.327,arunkumar.panchavarnam@aparinnosys.com,2021-10-04 17:40:07.810,true,11016,ID209011,arunkumar.panchavarnam@aparinnosys.com,2021-10-04 17:24:53.820,<br/> arunkumar.panchavarnam@aparinnosys.com w...
6,971196,735ab6,2021-10-05 17:11:23,Backroom door operated without authorization,Critical,2021-10-05 17:12:08.193,arunkumar.panchavarnam@aparinnosys.com,2021-10-05 17:27:28.667,true,11016,ID209011,arunkumar.panchavarnam@aparinnosys.com,2021-10-05 17:12:40.773,<br/> arunkumar.panchavarnam@aparinnosys.com w...
9,971201,735ab6,2021-10-05 17:12:50,ATM-1 Chest Door operated without authorization,Critical,2021-10-05 17:12:47.690,arunkumar.panchavarnam@aparinnosys.com,2021-10-05 17:27:51.250,true,11016,ID209011,arunkumar.panchavarnam@aparinnosys.com,2021-10-05 17:15:16.910,<br/> arunkumar.panchavarnam@aparinnosys.com w...


In [56]:
Events['TicketAge']=Events['ClosedOn']-Events['OpenedOn']
Events['AcknowledgeAge']=Events['UpdatedON']-Events['OpenedOn']
Events['TicketAge']=Events['TicketAge'].astype('timedelta64[m]')
Events['AcknowledgeAge']=Events['AcknowledgeAge'].astype('timedelta64[m]')

In [57]:
Events=Events.drop_duplicates(keep='first')
Events.shape

(8836, 17)

In [58]:
import numpy as np

In [59]:
conditions = [
    (Events['EventType'] == 'SOS') & (Events['TicketAge'] > 30),
    (Events['EventType'] == 'Critical') & (Events['TicketAge'] > 60),
    (Events['EventType'] == 'SOS') & (Events['AcknowledgeAge'] > 5),
    (Events['EventType'] == 'Critical') & (Events['AcknowledgeAge'] > 10)]

# create a list of the values we want to assign for each condition
values = ['Penalty Applicable','Penalty Applicable','Penalty Applicable','Penalty Applicable']

# create a new column and use np.select to assign values to it using our lists as arguments
Events['Penalty'] = np.select(conditions, values)

# display updated DataFrame
Events.head()

,EventId,McuId,AtTime,Msg,EventType,OpenedOn,ClosedBy,ClosedOn,qStatus,s_Id,s_Name,UpdatedBy,UpdatedON,Remarks,TicketAge,AcknowledgeAge,Penalty
1,967527,735ab6,2021-10-04 06:18:06,Backroom door operated without authorization,Critical,2021-10-04 06:17:34.570,prasanth.loganathan@aparinnosys.com,2021-10-04 06:22:33.933,true,11016,ID209011,prasanth.loganathan@aparinnosys.com,2021-10-04 06:19:20.263,<br/> prasanth.loganathan@aparinnosys.com working,4.0,1.0,0
2,967527,735ab6,2021-10-04 06:18:06,Backroom door operated without authorization,Critical,2021-10-04 06:17:34.570,prasanth.loganathan@aparinnosys.com,2021-10-04 06:22:33.933,true,11016,ID209011,prasanth.loganathan@aparinnosys.com,2021-10-04 06:19:20.360,<br/> prasanth.loganathan@aparinnosys.com working,4.0,1.0,0
4,969055,735ab6,2021-10-04 17:19:44,ATM-1 Chest Door operated without authorization,Critical,2021-10-04 17:19:10.327,arunkumar.panchavarnam@aparinnosys.com,2021-10-04 17:40:07.810,true,11016,ID209011,arunkumar.panchavarnam@aparinnosys.com,2021-10-04 17:24:53.820,<br/> arunkumar.panchavarnam@aparinnosys.com w...,20.0,5.0,0
6,971196,735ab6,2021-10-05 17:11:23,Backroom door operated without authorization,Critical,2021-10-05 17:12:08.193,arunkumar.panchavarnam@aparinnosys.com,2021-10-05 17:27:28.667,true,11016,ID209011,arunkumar.panchavarnam@aparinnosys.com,2021-10-05 17:12:40.773,<br/> arunkumar.panchavarnam@aparinnosys.com w...,15.0,0.0,0
9,971201,735ab6,2021-10-05 17:12:50,ATM-1 Chest Door operated without authorization,Critical,2021-10-05 17:12:47.690,arunkumar.panchavarnam@aparinnosys.com,2021-10-05 17:27:51.250,true,11016,ID209011,arunkumar.panchavarnam@aparinnosys.com,2021-10-05 17:15:16.910,<br/> arunkumar.panchavarnam@aparinnosys.com w...,15.0,2.0,0


In [60]:
Events['Penalty'].value_counts()

0                     8149
Penalty Applicable     687
Name: Penalty, dtype: int64

In [61]:
sos=Events[(Events['EventType']=='SOS') & (Events['Penalty']=='Penalty Applicable')]

In [62]:
sos.shape

(162, 17)

In [63]:
penalty=Events[Events['Penalty']=='Penalty Applicable']

In [64]:
penalty.shape

(687, 17)

In [66]:
penalty.to_csv('penalty.csv')